In [12]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from sklearn.utils import shuffle

In [13]:
data_path = r'/Users/alexandraszenti/Documents/GitHub Projects/ARRBHack-2018/_data'

In [18]:
df = pd.read_csv(os.path.join(data_path,'cleaned_crash_locations_all.csv'))

In [19]:
df.shape

(306558, 26)

## Analysing environments where cyclists are more likely to be in an accident

In order to predict when a cyclist is heading towards a situation where an incident is more likely to happen, we need to make some assumptions regarding the current data:
- Unknown data is not useful and if a feature has unkown fields we should exclude that from our model
- We don't have any information regards who has been hospitalised if there were more than one casualties in an accidents.. so we will assume that always the weaker role gets the more serious injures: pedestrian > cyclist > moped > car > bus / truck


In [20]:
df = shuffle(df)
df.head()

,Crash_Severity,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Loc_Suburb,Loc_ABS_Remoteness,Crash_Roadway_Feature,Crash_Traffic_Control,Crash_Speed_Limit,Crash_Road_Surface_Condition,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
284838,Minor injury,July,Tuesday,8,Trinity Beach,Outer regional,Intersection - Roundabout,Give way sign,60 km/h,Sealed - Dry,...,0,2,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0
208774,Property damage only,March,Saturday,23,Chermside,Major cities,Intersection - Cross,Operating traffic lights,60 km/h,Sealed - Wet,...,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
203346,Hospitalisation,December,Thursday,13,Mackay,Inner regional,Intersection - Cross,Give way sign,60 km/h,Sealed - Dry,...,0,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
129596,Property damage only,September,Wednesday,11,Cairns City,Outer regional,Intersection - Cross,Operating traffic lights,0 - 50 km/h,Sealed - Dry,...,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
160471,Minor injury,February,Sunday,22,Annerley,Major cities,No Roadway Feature,No traffic control,60 km/h,Sealed - Wet,...,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df.tail(20)

,Crash_Severity,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Loc_Suburb,Loc_ABS_Remoteness,Crash_Longitude_GDA94,Crash_Latitude_GDA94,Crash_Roadway_Feature,Crash_Speed_Limit,...,Crash_Atmospheric_Condition,Crash_Lighting_Condition,Crash_Road_Horiz_Align,Crash_Road_Vert_Align,Count_Casualty_Fatality,Count_Casualty_Hospitalised,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Unit_Pedestrian,Count_Unit_Bicycle
290215,Hospitalisation,December,Tuesday,16,Greenslopes,Major cities,153.044295,-27.498582,No Roadway Feature,60 km/h,...,Clear,Daylight,Curved - view open,Level,0,1,0,0,1.0,0.0
303412,Minor injury,January,Wednesday,15,Longreach,Very remote,144.259210,-23.437493,Intersection - Cross,0 - 50 km/h,...,Clear,Daylight,Curved - view open,Level,0,0,0,1,0.0,0.0
1721,Property damage only,February,Friday,3,Caboolture,Major cities,152.949010,-27.084622,No Roadway Feature,60 km/h,...,Raining,Darkness - Not lighted,Straight,Level,0,0,0,0,0.0,0.0
99609,Property damage only,June,Friday,10,Rochedale,Major cities,153.107294,-27.572057,Intersection - Interchange,100 - 110 km/h,...,Clear,Daylight,Straight,Level,0,0,0,0,0.0,0.0
37085,Property damage only,September,Sunday,18,Coominglah,Remote,150.848664,-24.765539,No Roadway Feature,100 - 110 km/h,...,Clear,Dawn/Dusk,Straight,Level,0,0,0,0,0.0,0.0
268308,Hospitalisation,March,Monday,1,Belmont,Major cities,153.122800,-27.531760,No Roadway Feature,100 - 110 km/h,...,Clear,Darkness - Lighted,Curved - view open,Level,0,1,0,0,0.0,0.0
191711,Minor injury,June,Monday,15,Verrierdale,Inner regional,152.994855,-26.476293,No Roadway Feature,80 - 90 km/h,...,Clear,Daylight,Straight,Crest,0,0,0,1,0.0,0.0
164101,Minor injury,March,Monday,9,Cannon Hill,Major cities,153.095897,-27.472590,Median Opening,60 km/h,...,Clear,Daylight,Straight,Level,0,0,0,2,0.0,0.0
278634,Medical treatment,January,Thursday,14,Kuranda,Outer regional,145.662065,-16.831452,No Roadway Feature,60 km/h,...,Clear,Daylight,Curved - view open,Grade,0,0,1,0,0.0,0.0
293658,Medical treatment,April,Sunday,10,Dicky Beach,Major cities,153.134640,-26.782818,Intersection - T-Junction,60 km/h,...,Clear,Daylight,Straight,Level,0,0,1,0,0.0,0.0


What do we want to count?

count_casualty_fatality + count_casualty_hospitalised + count_casualty_medcicallyTreated + count_casualty_minorInjury = count_unit_pedestrian + count_unit_bycicle + count_unit_bus + count_unit_other + count_unit_truck + count_unit_moped + count_unit_car ???? 

1 or 0  - Did the cyclist have an FSI? 


1 if cyclist was in fatality or hospitalised and no pedestrian was in fatality or hospitalised

0 if more pedestrians were involved and no fsi remains for cyclist = (fatality + hospitalised) <= count_pedestrian
OR
Cyclist